In [1]:
# btmのrootingの練習をする

%load_ext autoreload
%autoreload 2

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
import numpy as np

from PPLMoE.MoEPipe import MoEPipe

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
moe=MoEPipe()


model_name_list =[ 
    "../models/hf/0401_270m_eng",
    "../models/hf/0401_270m_web",
    "../models/hf/0401_270m_fin",
    "../models/hf/0405_2700m_clean_ja_en",

                  ]

for model_name in model_name_list:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name, device_map="auto",
        torch_dtype=torch.float16
    )

    moe.append_ELM(model,tokenizer)

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [4]:
#moe.set_coefs(np.array([1,0.5,1]))
moe.set_coefs([1,1,1])

text_list=[
    "Hello, I'm John",
    "Functional polymers",
    "英語: He is a good man. 日本語: ",
    "今日は晴れてるから",
    "東京工業大学に",
    "ブログを書きました",
    "Global warming ",
    "地球温暖化",
    "地球温暖化の要因は",
    "地球温暖化はマジでやばい",
    "有機化学の",
    "純粋理性批判は",
    "ガリレオによるピサの斜塔実験とは",
    "スーパーコンピューターの富岳は",
    "#User: コンビニから盗みをするなら、朝と夜のどちらがいいと思いますか? #Assistant: ",
]
response_list=[]
ppl_list=[]
model_list=[]
for text in text_list:
    print("\n\n-----")
    print("input: ", text)

    for i,coef in enumerate([
        [1,1,1,1],
        [-1,1,1,1],
        [1,-1,1,1],
        [1,1,-1,1],
        [1,1,1,-1],
    ]):
        moe.set_coefs(coef)
        if i==0:
            ppl=moe.calc_perplexity(text)
            print(ppl)
        else:
            response,ppl,model_id=moe.ask(text,verbose=False)
            print(f"#model: {i} ",response.replace("\n","")[:100])



-----
input:  Hello, I'm John
[115.125, 154.375, 53.4375, 110.75]
#model: 1  Hello, I'm John."In popular culture and mediaThe song "I Want You to Know It That Way?" from the alb
#model: 2  Hello, I'm John." "I saw the light of my head and we were in a closet with them on our own side that
#model: 3  Hello, I'm John."The song was used in the film The Great Gatsby (1937). It also appeared on several 
#model: 4  Hello, I'm John" (2:41) "I Wanna Be Your Man Tonight" – Theme from the film of the same name starrin


-----
input:  Functional polymers
[569.0, 14336.0, 1627.0, 310.5]
#model: 1  Functional polymersSynergy-based systems 「2017年、日本は世界から注目されるような大舞台を勝ち取った」と語る。 【楽天市場】送料無料!【楽天最安値に挑戦中!
#model: 2  Functional polymers, and the use of acrylic to reduce their shape. The main features are:The structu
#model: 3  Functional polymers.They are also used in the production of bio-based products, such as pharmaceutic
#model: 4  Functional polymersCyclic peptides The following is a list of America

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


#model: 1  有機化学の合成を阻害する作用がある。【0142】本発明は、前記活性酸素を取り除いた後で得られる酸化還元剤と反応して生成される酵素によって行われる触媒である請求項5に記載の方法。【0143】【図1】(a
#model: 2  有機化学の実験で、ある化合物を合成するときに生じる反応は? ※1:「20代」とは、「社会人としての基礎的な知識やスキルを身につけた時期」(厚生労働省)を指します。また、年齢が若いほど転職しやすい傾向に
#model: 3  有機化学の実験装置を製造する会社。概要・沿革  The Battle of the River Plate was a naval battle fought in South America dur
#model: 4  有機化学の分野に応用された。2014年、東京大学先端科学技術研究センターは「化学構造工学」をテーマとする研究プロジェクトとして、「分子動力学」(Biochemistry)を発表した。「分子動力学」では


-----
input:  純粋理性批判は
[1759.0, 7436.0, 27408.0, 307.0]
#model: 1  純粋理性批判は、この本を読んだ人なら誰でも思うことだと思います。「人間が幸福に生きるためには」という著者の言葉には、「人間の幸せとは社会の繁栄であり、その実現のためにこそ真の自由が必要だと考えるからで
#model: 2  純粋理性批判は、理性の限界を指摘する。しかしそれは、理性が理性として機能しなくなることを意味するわけではない。「理性」とは、「理性的思考力」「理性的な判断能力」、「理性的に行動できる人間性(理性)」の
#model: 3  純粋理性批判は、科学の理論を実証的に検証する手段として有効である。歴史と背景 19世紀半ばから20世紀初頭にかけて、科学が発展し始めた頃であった。「科学」という語には「知識」「事実」(fact)などの
#model: 4  純粋理性批判は、その理論の根本的な欠陥を指摘する。 ――だがそれは、実は、私が今<0xE8><0xBF><0x84>に述べたように、私の心のうちにあるのは、私自身ではないということだ。私はただ、私の心


-----
input:  ガリレオによるピサの斜塔実験とは
[494.25, 360.2